In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

# data1 = pd.read_csv('train.csv',header=0)
def plot_feature_importance(clf, feature_names):
    feature_importance = clf.feature_importances_
    # make importances relative to max importance
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)[-10:]
    pos = np.arange(sorted_idx.shape[0]) + 4.5
    plt.subplot(1, 2, 2)
    plt.barh(pos, feature_importance[sorted_idx],height=0.3,align='center')
    # plt.yticks(pos, feature_names[sorted_idx])
    plt.yticks(pos, [feature_names[idx] for idx in sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()

    
def get_result(conf):
    print(conf)
    recall = conf[0][0] / conf[0].sum() * 100
    precision = conf[0][0] / conf[:, 0].sum() * 100
    accuracy = (conf[0][0] + conf[1][1] + conf[2][2]) / conf.sum() * 100
    F1 = 2 * recall * precision / (recall + precision)
    print('recall : %.4f %%' % recall)
    print('precision : %.4f %%' % precision)
    print('accuracy : %.4f %%' % accuracy)
    print('F1 : %.4f' % F1)
    print([recall, precision, accuracy, F1])
    return [recall, precision, accuracy, F1]

In [2]:
class tree:
    def __init__(self, feature_list):
        self.feature_list = feature_list
        self.data_train = pd.read_csv('train_ne_three.csv')
        self.data_test = pd.read_csv('test_ne_three.csv')
#         self.data_val = pd.read_csv('val.csv')
    def decision_result(self):
        print('decision_tree')
        return DecisionTreeClassifier(max_depth =10,min_samples_split=40,min_samples_leaf=10)
    def random_result(self):
        print('random_forest')
        return RandomForestClassifier(n_estimators=300,max_depth =10,min_samples_split=40,min_samples_leaf=10)
    def adaboost_result(self):
        print('adaboost')
        return AdaBoostClassifier(DecisionTreeClassifier(max_depth=10, min_samples_split=40, min_samples_leaf=10),
                         algorithm="SAMME",
                         n_estimators=300, learning_rate=0.8)
    def process(self, model):
        result_ABC = []
        for feature in self.feature_list:
#             rfc = joblib.load("canada.model")
            train_data = self.data_train[feature].values
            train_label = self.data_train['label'].values - 1
#             data1 = self.data_test.loc[self.data_test['label'] == 1]
#             data2 = self.data_test.loc[self.data_test['label'] == 2]
#             data = pd.concat([data1,data2])
            test_data = self.data_test[feature].values
            test_label = self.data_test['label'].values - 1
#             val_data = self.data_val[feature].values
#             val_label = self.data_val['label'].values - 1
            rfc = model.fit(train_data,train_label)
            joblib.dump(rfc, "ne_shpindex.model")
#             my_plots = plot_partial_dependence(rfc, 
#                                    features=[0], 
#                                    X=train_data, 
#                                    feature_names=list_test)
            prediction = rfc.predict(test_data)
            conf = confusion_matrix(test_label, prediction)
            result = get_result(conf)
            result_ABC.append(result)
            plot_feature_importance(clf=rfc, feature_names=feature)
#             val = rfc.predict(val_data)
#             conf_val = confusion_matrix(val, val_label)
#             result_val = get_result(conf_val)
        return result_ABC

In [3]:
list_01 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',                
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',                
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','PR_NDVI_1','PR_NDVI_2','TXU_ENT','TXU_asm','area','LT_mag',                
            'LT_dur','TXU_HOM','TXU_CON']

list_02 = ['PT_NDMI_0','PR_NDMI_1','PR_NDMI_2','PT_NBR_0','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PR_NDVI_1','PR_NDVI_2','TXU_ENT',                
            'TXU_asm','area','LT_mag', 'LT_dsnr','LT_dur','TXU_HOM','TXU_CON','ndvi_rvalue','ndmi_rvalue','nbr_rvalue',                
            'ndvi_slope','ndmi_slope','nbr_slope']

list_05 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PT_NBR_0',                
           'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PT_NDVI_0','PT_NDVI_1',                
           'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5', 'D/A', 'Shape Index', 'Perimeter'
            ,'Area','LT_mag','NDVI_rvalue','NDMI_rvalue','NBR_rvalue','NDVI_slope','NDMI_slope','NBR_slope']

list_06 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PT_NBR_0',\
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PT_NDVI_0','PT_NDVI_1',\
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','area','LT_mag','TXU_asm','TXU_HOM',\
            'TXU_ENT','TXU_CON','Area','Perimeter','D/A','Perimeter',\
           'Shape Index','ndmi_rvalue','ndvi_rvalue','nbr_rvalue',\
           'ndvi_slope','ndmi_slope','nbr_slope']

list_test = ['PT_NDMI_0','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PT_NDVI_5'      
            ,'area','LT_mag','NDMI_rvalue','NBR_rvalue']

# feature_list = [list_00, list_01, list_02]
feature_list = [list_05]

In [4]:
tree = tree(feature_list)
boost = tree.process(tree.adaboost_result())

adaboost


KeyError: "['area'] not in index"